In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import anndata
import scanpy as sc
from ALLCools.mcds import MCDS
from ALLCools.clustering import tsne, significant_pc_test, filter_regions, remove_black_list_region, lsi, binarize_matrix
from ALLCools.plot import *
from ALLCools.clustering import ConsensusClustering, Dendrogram, get_pc_centers

### this notebook generate .phy files used in Iqtree that construct lineage tree with methylation data.
### formate the .phy the to the following structure: 

find the tutorial here: http://www.iqtree.org/doc/Tutorial 

- the first number is the number of cells, the second is the length of sequence
- 1 means hypomethylation (from 5kb bin) - means gap (ignor gap bere)

4 6  
S1   010101   
S2   110011  
S3   0--100  
S4   10--10  

In [199]:
import numpy as np
metadata = pd.read_csv('../Chaligne2021NG/Chaligne2021NG/metadata.tsv', sep=',', index_col = 0)
mcds = MCDS.open('../Chaligne2021NG/Chaligne2021NG/Chaligne2021NG.mcds', var_dim='chrom100k')

# mcds = MCDS.open('/gale/netapp/entex/GBM/*/matrix/*.mcds', var_dim='chrom5k')

In [29]:
mcds.add_cell_metadata(metadata)
selc = metadata.index
mcds = mcds.sel(cell = mcds.get_index('cell').intersection(selc))
#mcds = mcds.remove_black_list_region(black_list_path='/gale/netapp/home/hanliu/ref/blacklist/hg38-blacklist.v2.bed.gz')
chrom_to_remove = ['chrL', 'chrM', 'chrX', 'chrY','chr6','chr7','chr9','chr10','chr19','chr22']
mcds = mcds.remove_chromosome( chrom_to_remove, 'chrom100k')
mcds

9256 chrom100k features in ['chrL', 'chrM', 'chrX', 'chrY', 'chr6', 'chr7', 'chr9', 'chr10', 'chr19', 'chr22'] removed.


<xarray.MCDS>
Dimensions:                  (cell: 50, chrom100k: 21639, count_type: 2)
Coordinates:
  * cell                     (cell) object 'GBM_MGH105C_A1_S1_L008.CCTTCG' .....
  * chrom100k                (chrom100k) <U10 'chr1_0' 'chr1_1' ... 'chr8_1451'
    chrom100k_chrom          (chrom100k) <U5 dask.array<chunksize=(21639,), meta=np.ndarray>
    chrom100k_end            (chrom100k) int64 dask.array<chunksize=(21639,), meta=np.ndarray>
    chrom100k_start          (chrom100k) int64 dask.array<chunksize=(21639,), meta=np.ndarray>
  * count_type               (count_type) <U3 'mc' 'cov'
    mc_type                  <U3 'CGN'
    cell_# reads (x10^6)     (cell) float64 1.549 3.295 2.308 ... 1.962 1.997
    cell_# CpGs (x10^6)      (cell) float64 0.07586 0.1791 ... 0.1327 0.1297
    cell_CG conversion rate  (cell) float64 99.8 99.8 99.8 ... 99.8 99.8 99.8
    cell_Sample              (cell) object 'MGH105C' 'MGH105C' ... 'MGH105D'
    cell_Type                (cell) object 'GBM' 'GBM' 'GBM' ... 'GBM' 'GBM'
Data variables:
    chrom100k_da             (cell, chrom100k, count_type) uint32 dask.array<chunksize=(50, 4841, 2), meta=np.ndarray>
Attributes:
    obs_dim:  cell
    var_dim:  chrom100k

In [30]:
mcds['chrom100k_da'].to_pandas()

ValueError: cannot convert arrays with 3 dimensions into pandas objects

In [14]:
# mcds = mcds.remove_black_list_region(black_list_path='/gale/netapp/home/hanliu/ref/blacklist/hg38-blacklist.v2.bed.gz')
# chrom_to_remove = ['chrL', 'chrM', 'chrX', 'chrY']
# mcds = mcds.remove_chromosome( chrom_to_remove,'chrom5k')
# mcds

45816 chrom5k features removed due to overlapping (bedtools intersect -f 0.2) with black list regions.
34491 chrom5k features in ['chrL', 'chrM', 'chrX', 'chrY'] removed.


<xarray.MCDS>
Dimensions:                    (cell: 1153, chrom5k: 537362)
Coordinates: (12/106)
  * cell                       (cell) object 'CW_2209_1-CW_2209_2-A10-AD001' ...
  * chrom5k                    (chrom5k) <U11 'chr1_159' ... 'chr9_27643'
    chrom5k_chrom              (chrom5k) <U5 dask.array<chunksize=(537362,), meta=np.ndarray>
    chrom5k_end                (chrom5k) int64 dask.array<chunksize=(537362,), meta=np.ndarray>
    chrom5k_start              (chrom5k) int64 dask.array<chunksize=(537362,), meta=np.ndarray>
    cell_Plate                 (cell) object 'CW_2209_1' ... 'CW_2209_4'
    ...                         ...
    cell_chr19_3c              (cell) float64 -0.5824 -1.224 ... -1.098 0.1056
    cell_chr20_3c              (cell) float64 -0.2391 -1.021 ... -0.9967 -0.9315
    cell_chrX_3c               (cell) float64 -0.3159 0.4379 ... 0.7702 0.4699
    cell_chr21_3c              (cell) float64 -2.297 -2.255 ... -2.302 -1.594
    cell_chr22_3c              (cell) float64 -2.286 -1.543 ... -1.988 -1.038
    cell_type                  (cell) object 'non-malignant' ... 'malignant'
Data variables:
    chrom5k_da_CGN-hypo-score  (cell, chrom5k) float16 dask.array<chunksize=(68, 537362), meta=np.ndarray>
Attributes:
    obs_dim:  cell
    var_dim:  chrom5k

In [18]:
df = mcds['chrom5k_da_CGN-hypo-score'].to_pandas()

In [19]:
df = df.round().astype(int)
df

chrom5k,chr1_0,chr1_1,chr1_2,chr1_3,chr1_4,chr1_5,chr1_6,chr1_7,chr1_8,chr1_9,...,chr8_29018,chr8_29019,chr8_29020,chr8_29021,chr8_29022,chr8_29023,chr8_29024,chr8_29025,chr8_29026,chr8_29027
cell,,,,,,,,,,,,,,,,,,,,,
GBM_MGH105C_A1_S1_L008.CCTTCG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GBM_MGH105C_C1_S2_L008.ATCGAC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GBM_MGH105C_C1_S2_L008.GCATTC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GBM_MGH105C_C1_S2_L008.GTTGAG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GBM_MGH105C_E1_S3_L008.TGCTGC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GBM_MGH105C_G1_S4_L008.GTGAGG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GBM_MGH105C_G1_S4_L008.TGCTGC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GBM_MGH105C_G1_S4_L008.TGTAGG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MGH105A_P7_A1_S1_L001.ACTCAC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
df.iloc[4,:].sum()

2106

In [19]:
import glob
cells = glob.glob('../CNV_analysis/bedfiles/cw_2209/*')
cells

['../CNV_analysis/bedfiles/cw_2209/CW_2209_1-CW_2209_2-A10-AD008.bed.gz',
 '../CNV_analysis/bedfiles/cw_2209/CW_2209_1-CW_2209_2-A10-AD012.bed.gz',
 '../CNV_analysis/bedfiles/cw_2209/CW_2209_1-CW_2209_2-A11-AD001.bed.gz',
 '../CNV_analysis/bedfiles/cw_2209/CW_2209_1-CW_2209_2-B10-AD004.bed.gz',
 '../CNV_analysis/bedfiles/cw_2209/CW_2209_1-CW_2209_2-B10-AD007.bed.gz',
 '../CNV_analysis/bedfiles/cw_2209/CW_2209_1-CW_2209_2-B10-AD008.bed.gz',
 '../CNV_analysis/bedfiles/cw_2209/CW_2209_1-CW_2209_2-B10-AD010.bed.gz',
 '../CNV_analysis/bedfiles/cw_2209/CW_2209_1-CW_2209_2-B11-AD001.bed.gz',
 '../CNV_analysis/bedfiles/cw_2209/CW_2209_1-CW_2209_2-C10-AD002.bed.gz',
 '../CNV_analysis/bedfiles/cw_2209/CW_2209_1-CW_2209_2-C10-AD004.bed.gz',
 '../CNV_analysis/bedfiles/cw_2209/CW_2209_1-CW_2209_2-C10-AD007.bed.gz',
 '../CNV_analysis/bedfiles/cw_2209/CW_2209_1-CW_2209_2-C10-AD010.bed.gz',
 '../CNV_analysis/bedfiles/cw_2209/CW_2209_1-CW_2209_2-C10-AD012.bed.gz',
 '../CNV_analysis/bedfiles/cw_2209/CW_

In [41]:
cell_index = [ c.split('/')[-1].split('.')[0] for c in cells]

In [51]:
sum(~df.columns.str.contains('chr7|chr10|chr19', regex=True))

469227

In [ ]:
#remove chr7 10 19, which have cnv
df.loc[cell_index, ~df.columns.str.contains('chr7|chr10|chr19', regex=True)]

In [52]:
cell_df = df.loc[cell_index, ~df.columns.str.contains('chr7|chr10|chr19', regex=True)]
cell_df

chrom5k,chr1_159,chr1_160,chr1_161,chr1_162,chr1_163,chr1_164,chr1_165,chr1_166,chr1_167,chr1_168,...,chr9_27634,chr9_27635,chr9_27636,chr9_27637,chr9_27638,chr9_27639,chr9_27640,chr9_27641,chr9_27642,chr9_27643
cell,,,,,,,,,,,,,,,,,,,,,
CW_2209_1-CW_2209_2-A10-AD008,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CW_2209_1-CW_2209_2-A10-AD012,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CW_2209_1-CW_2209_2-A11-AD001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CW_2209_1-CW_2209_2-B10-AD004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
CW_2209_1-CW_2209_2-B10-AD007,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
CW_2209_1-CW_2209_2-B10-AD008,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
CW_2209_1-CW_2209_2-B10-AD010,0,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
CW_2209_1-CW_2209_2-B11-AD001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CW_2209_1-CW_2209_2-C10-AD002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0


In [26]:
df = df.astype(str)

In [27]:
''.join(df.iloc[1,:].values[:50])

'00000000000000000000000000000000000000000000000000'

In [31]:
# use 5kb methylation score this time
with open('MGH105.phy', 'w') as f:
    f.write(str(len(df.index)) + ' ' + str(len(df.iloc[0,:])) + '\n')
    for i in df.index:
        f.write(i+'\t')
        f.write(''.join(df.loc[i,:].values))
        f.write('\n')
    f.close()

## select cells from Chaligne data

/gale/netapp/cemba3c/projects/SingleCellMethylomes/Chaligne2021NG/raw/

In [101]:
a = pd.read_csv('/gale/netapp/cemba3c/projects/SingleCellMethylomes/Chaligne2021NG/raw/MGH105A.csv', index_col = 0)
a

,MGH105A_P7_A1_S1_L001.ACAACC.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_A1_S1_L001.ACTCAC.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_A1_S1_L001.AGGATG.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_A1_S1_L001.ATCAAG.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_A1_S1_L001.ATCGAC.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_A1_S1_L001.CAAGAG.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_A1_S1_L001.CATGAC.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_A1_S1_L001.CGGTAG.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_A1_S1_L001.CTATTG.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_A1_S1_L001.CTCAGC.R1.fastq_bismark_bt2_pe.bismark.cov,...,MGH105A_P7_G1_S4_L001.CTATTG.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_G1_S4_L001.CTCAGC.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_G1_S4_L001.GAAGTC.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_G1_S4_L001.GGTAAC.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_G1_S4_L001.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_G1_S4_L001.GTTGAG.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_G1_S4_L001.TATCTC.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_G1_S4_L001.TGACAG.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_G1_S4_L001.TGCTGC.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_G1_S4_L001.TGTAGG.R1.fastq_bismark_bt2_pe.bismark.cov
site,,,,,,,,,,,,,,,,,,,,,
chr1-0000010525,-,-,-,-,1,-,-,-,-,-,...,-,-,-,-,-,1,-,-,-,-
chr1-0000010563,-,-,-,-,1,-,-,-,-,-,...,-,-,-,-,-,1,-,-,-,-
chr1-0000010571,-,-,-,-,1,-,-,-,-,-,...,-,-,-,-,-,1,-,-,-,-
chr1-0000010577,-,-,-,-,1,-,-,-,-,-,...,-,-,-,-,-,1,-,-,-,-
chr1-0000010579,-,-,-,-,1,-,-,-,-,-,...,-,-,-,-,-,1,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chrY-0057200821,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
chrY-0057201662,-,-,-,-,-,-,-,-,-,-,...,-,-,0,-,-,-,-,-,-,-
chrY-0057201684,-,-,-,-,-,-,-,-,-,-,...,-,-,0,-,-,-,-,-,-,-


In [93]:
cells = tree.get_tip_labels()
cells = set([c+'.R1.fastq_bismark_bt2_pe.bismark.cov' for c in cells])

In [102]:
a = a[a.columns.intersection(cells)]
a

,MGH105A_P7_A1_S1_L001.ACTCAC.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_A1_S1_L001.CAAGAG.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_C1_S2_L001.GAAGTC.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_C1_S2_L001.GGTAAC.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_C1_S2_L001.TCTCTG.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_G1_S4_L001.CCTTCG.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_G1_S4_L001.CTATTG.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_G1_S4_L001.GGTAAC.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_G1_S4_L001.TATCTC.R1.fastq_bismark_bt2_pe.bismark.cov
site,,,,,,,,,
chr1-0000010525,-,-,-,-,-,-,-,-,-
chr1-0000010563,-,-,-,-,-,-,-,-,-
chr1-0000010571,-,-,-,-,-,-,-,-,-
chr1-0000010577,-,-,-,-,-,-,-,-,-
chr1-0000010579,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...
chrY-0057200821,-,-,-,-,-,-,-,-,-
chrY-0057201662,-,-,-,-,-,-,-,-,-
chrY-0057201684,-,-,-,-,-,-,-,-,-


In [103]:
for i in ['B','C','D']:
    tmp = pd.read_csv(f'/gale/netapp/cemba3c/projects/SingleCellMethylomes/Chaligne2021NG/raw/MGH105{i}.csv', index_col = 0)
    tmp = tmp[tmp.columns.intersection(cells)]
    a = a.join(tmp,how='inner')
    print(f'finished {i}')

finished B
finished C
finished D


In [161]:
a

,MGH105A_P7_A1_S1_L001.ACTCAC.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_A1_S1_L001.CAAGAG.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_C1_S2_L001.GAAGTC.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_C1_S2_L001.GGTAAC.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_C1_S2_L001.TCTCTG.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_G1_S4_L001.CCTTCG.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_G1_S4_L001.CTATTG.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_G1_S4_L001.GGTAAC.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105A_P7_G1_S4_L001.TATCTC.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105B_P6_A1_S1_L005.ACAACC.R1.fastq_bismark_bt2_pe.bismark.cov,...,GBM_MGH105C_G1_S4_L008.TGTAGG.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105D_P7_A1_S1_L004.ACTCAC.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105D_P7_A1_S1_L004.CATGAC.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105D_P7_A1_S1_L004.CCATAG.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105D_P7_A1_S1_L004.TCTCTG.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105D_P7_C1_S2_L004.ATCAAG.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105D_P7_C1_S2_L004.GCATTC.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105D_P7_E1_S3_L004.ACAACC.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105D_P7_E1_S3_L004.GAAGTC.R1.fastq_bismark_bt2_pe.bismark.cov,MGH105D_P7_E1_S3_L004.GTTGAG.R1.fastq_bismark_bt2_pe.bismark.cov
site,,,,,,,,,,,,,,,,,,,,,
chr1-0000010525,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,1,-,-,-,-,-
chr1-0000010563,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,1,-,-,-,-,-
chr1-0000010571,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,1,-,-,-,-,-
chr1-0000010577,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,1,-,-,-,-,-
chr1-0000010579,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,1,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chrY-0057047520,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
chrY-0057070424,-,-,1,-,1,-,-,0,1,-,...,0,-,-,1,-,-,-,-,-,-
chrY-0057094497,-,-,-,-,1,-,-,-,-,-,...,-,1,1,-,-,-,-,1,-,-


In [104]:
b = a[~a.index.str.contains('chr6|chr7|chr9|chr10|chr19|chr22',regex=True)]
b = b.T
b

site,chr1-0000010525,chr1-0000010563,chr1-0000010571,chr1-0000010577,chr1-0000010579,chr1-0000026803,chr1-0000026810,chr1-0000026837,chr1-0000076995,chr1-0000121488,...,chrY-0056887089,chrY-0056908781,chrY-0057047038,chrY-0057047042,chrY-0057047062,chrY-0057047520,chrY-0057070424,chrY-0057094497,chrY-0057094523,chrY-0057094537
MGH105A_P7_A1_S1_L001.ACTCAC.R1.fastq_bismark_bt2_pe.bismark.cov,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
MGH105A_P7_A1_S1_L001.CAAGAG.R1.fastq_bismark_bt2_pe.bismark.cov,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
MGH105A_P7_C1_S2_L001.GAAGTC.R1.fastq_bismark_bt2_pe.bismark.cov,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,1,-,-,-
MGH105A_P7_C1_S2_L001.GGTAAC.R1.fastq_bismark_bt2_pe.bismark.cov,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
MGH105A_P7_C1_S2_L001.TCTCTG.R1.fastq_bismark_bt2_pe.bismark.cov,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,1,1,-,-
MGH105A_P7_G1_S4_L001.CCTTCG.R1.fastq_bismark_bt2_pe.bismark.cov,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
MGH105A_P7_G1_S4_L001.CTATTG.R1.fastq_bismark_bt2_pe.bismark.cov,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
MGH105A_P7_G1_S4_L001.GGTAAC.R1.fastq_bismark_bt2_pe.bismark.cov,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,0,-,-,-
MGH105A_P7_G1_S4_L001.TATCTC.R1.fastq_bismark_bt2_pe.bismark.cov,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,1,-,-,-
MGH105B_P6_A1_S1_L005.ACAACC.R1.fastq_bismark_bt2_pe.bismark.cov,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


In [200]:
b = pd.read_csv('/gale/netapp/cemba3c/projects/SingleCellMethylomes/Chaligne2021NG/raw/MGH107.csv',index_col = 0)
b = b[~b.index.str.contains('chr8|chr7',regex=True)]
b = b.T
b

site,chr1-0000010525,chr1-0000010563,chr1-0000010571,chr1-0000010577,chr1-0000010579,chr1-0000010589,chr1-0000010609,chr1-0000026803,chr1-0000026810,chr1-0000026837,...,chrY-0057070939,chrY-0057094497,chrY-0057094523,chrY-0057094537,chrY-0057122920,chrY-0057122924,chrY-0057147380,chrY-0057147382,chrY-0057147388,chrY-0057147405
MGH107_P8_A1_S1_L007.ACAACC.R1.fastq_bismark_bt2_pe.bismark.cov,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
MGH107_P8_A1_S1_L007.ACAGAC.R1.fastq_bismark_bt2_pe.bismark.cov,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
MGH107_P8_A1_S1_L007.ACTCAC.R1.fastq_bismark_bt2_pe.bismark.cov,1,1,1,1,1,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
MGH107_P8_A1_S1_L007.ATCAAG.R1.fastq_bismark_bt2_pe.bismark.cov,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
MGH107_P8_A1_S1_L007.ATCGAC.R1.fastq_bismark_bt2_pe.bismark.cov,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MGH107_P8_G1_S4_L007.GGTAAC.R1.fastq_bismark_bt2_pe.bismark.cov,1,-,-,-,1,1,1,-,-,-,...,-,-,-,-,-,-,-,-,-,-
MGH107_P8_G1_S4_L007.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
MGH107_P8_G1_S4_L007.GTTGAG.R1.fastq_bismark_bt2_pe.bismark.cov,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
MGH107_P8_G1_S4_L007.TATCTC.R1.fastq_bismark_bt2_pe.bismark.cov,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


In [201]:
with open('MGH107_chaligne.phy', 'w') as f:
    f.write(str(len(b.index)) + ' ' + str(len(b.iloc[0,:])) + '\n')
    for i in b.index:
        f.write(i+'\t')
        f.write(''.join(b.loc[i,:].values))
        f.write('\n')
    f.close()

### example command line

`iqtree -s cw_2209.phy -st BIN -m TESTNEW -mset GTR2 -nt AUTO -bb 1000 -wbtl -abayes -alrt 1000 -nstop 500 -wsr -nm 2000 -v -alninfo`

`/iqtree-1.6.9-Linux/bin/iqtree -s ./my_sample_name.phy -st BIN -m TESTNEW -mset GTR2 -nt AUTO -bb 1000 -wbtl -abayes -alrt 1000 -nstop 500 -wsr -nm 2000 -v -alninfo`

`-s` is filet name
`-m` is the option to specify the model name to use during the analysis. 
We opted for the new model selection procedure64 (`-m TESTNEW`), which additionally implements the FreeRate heterogeneity model inferring the site rates directly from the data

`-mset GTR2` General time reversible model ‘GTR2’ consistently outperformed the other model tested (Jukes–Cantor type model) for our methylation binary data

`Q-TREE` can utilize multiple CPU cores to speed up the analysis. A complement option -T (or -nt for version 1.x) allows specifying the number of CPU cores to use. 

set an upper limit of CPU cores that can automatically be assigned. Use the -ntmax option to do so

`-bb` or `-B` specifies the number of bootstrap replicates where 1000 is the minimum number recommended. # for version 1.x change -B to -bb

`-alrt` specifies the number of bootstrap replicates for SH-aLRT where 1000 is the minimum number recommended.

IQ-TREE also supports other tests such as the aBayes test 

read more: http://www.iqtree.org/doc/Tutorial 

In [ ]:
iqtree -s MGH105.phy -st BIN -m TESTNEW -mset GTR2 -nt AUTO -bb 1000 -wbtl -abayes -alrt 1000 -nstop 500 -wsr -nm 2000 -v -alninfo

In [69]:
tree.ntips

62

In [70]:
#tree.get_tip_labels()
#tree.nnodes
types = ['cancer']*40 +['normal']*(tree.ntips - 40)
colorlist = ["#d6557c" if "normal" in tip else "#5384a3" for tip in types]
tree.draw(
    tip_labels_align=True,
    tip_labels_colors=colorlist
);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="1000.0px" viewBox="0 0 500.0 1000.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t2407ff3e71e54749938e0b751c6b9289"> MGH124_CD45_P7_G1_S4_L003.CCATAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_C1_S2_L003.AGGATG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.TGCTGC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_G1_S4_L003.ACAACC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_G1_S4_L003.TGCTGC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_A1_S1_L003.ACTCAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_A1_S1_L003.CATGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_G1_S4_L003.TCTCTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_A1_S1_L003.AGAAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_A1_S1_L003.GCATTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_C1_S2_L003.ACAACC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_C1_S2_L003.CATGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_A1_S1_L003.CGGTAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_G1_S4_L003.CGGTAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.CTCAGC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_C1_S2_L003.TGTAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_C1_S2_L003.TGCTGC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_G1_S4_L003.GTTGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_A1_S1_L003.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_G1_S4_L003.TGTAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.CATGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.GCATTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_C1_S2_L003.CCATAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_C1_S2_L003.TATCTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_C1_S2_L003.CAAGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_A1_S1_L003.GAAGTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_G1_S4_L003.CTATTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_G1_S4_L003.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_G1_S4_L003.GCATTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_G1_S4_L003.CCTTCG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.TGTAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_A1_S1_L003.TCTCTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.CGGTAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.GTTGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.GGTAAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.ACAGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_A1_S1_L003.CTATTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.CCTTCG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_G1_S4_L003.CATGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.ACTCAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_G1_S4_L003.ACTCAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.ACAACC.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_A1_S1_L003.TGACAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_C1_S2_L003.CCTTCG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.CCATAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.CTATTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_E1_S3_L003.ATCAAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_C1_S2_L003.CTATTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH124_CD45_P7_G1_S4_L003.AGAAGG.R1.fastq_bismark_bt2_pe.bismark.cov M

## replace the tree with iqtree generated phylogeny tree (exclude cnv chrm)

replace the color scheme with cell type, clusters, or cnv inference

In [202]:
import toytree
#cw_2209
newick = ''
with open('MGH107_chaligne.phy.parstree', 'r') as f:
    newick = f.read()

tree = toytree.tree(newick)
canvas, axes, mark = tree.draw(use_edge_lengths=False)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="1000.0px" viewBox="0 0 500.0 1000.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t4b2bcb8906d645bcad249d07152af3df"> MGH107_P8_G1_S4_L007.ATCAAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_G1_S4_L007.ACAACC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_G1_S4_L007.CCTTCG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.GGTAAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_A1_S1_L007.ATCAAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_C1_S2_L007.TGTAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.CTATTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_C1_S2_L007.ATCAAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_G1_S4_L007.GGTAAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.AGAAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_G1_S4_L007.AGGATG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_G1_S4_L007.CAAGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.CGGTAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_G1_S4_L007.TGTAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_A1_S1_L007.ACAGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_C1_S2_L007.GGTAAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_A1_S1_L007.TCTCTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.TCTCTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_A1_S1_L007.CAAGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_G1_S4_L007.GTTGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.GCATTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.TATCTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_A1_S1_L007.GCATTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_G1_S4_L007.ACAGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.CCATAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_A1_S1_L007.GTTGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_C1_S2_L007.TATCTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_G1_S4_L007.TATCTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.AGGATG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_A1_S1_L007.ATCGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_A1_S1_L007.TGCTGC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_C1_S2_L007.GTTGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.ATCGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_A1_S1_L007.TGACAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_C1_S2_L007.AGGATG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_C1_S2_L007.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.TGTAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_A1_S1_L007.CCTTCG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_C1_S2_L007.CTCAGC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_A1_S1_L007.ACAACC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_G1_S4_L007.CTCAGC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_C1_S2_L007.ACAGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_C1_S2_L007.CAAGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_A1_S1_L007.TGTAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.CAAGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.ACAGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_G1_S4_L007.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_A1_S1_L007.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.ATCAAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_C1_S2_L007.GCATTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_G1_S4_L007.ACTCAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_C1_S2_L007.CCATAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.CATGAC.R1.fastq_bismark_bt2_pe.bismark

In [204]:
cellstates  = pd.read_csv('../Chaligne2021NG/IDHcellstates.csv', index_col = 0)
cellstates

,G1/S,G2/M,AC (PCA-only),AC (PCA+mice),OC (PCA-only),OC (PCA+mice),stemness,CellAssignment,Sample,DNAmeMatchingCell
Cell,,,,,,,,,,
MGH107_Plate12_RNA_ofRRBS_5433.13_S13_L005_R1_001,0.294109,0.197272,0.697301,0.545616,-0.605058,-0.382264,-0.573826,AC-like,MGH107,MGH107_P8_A1_S1_L007.GTTGAG
MGH107_Plate12_RNA_ofRRBS_5433.27_S27_L005_R1_001,-0.099435,0.152367,0.620014,0.597382,-0.346065,-0.462044,-0.603494,AC-like,MGH107,MGH107_P8_C1_S2_L007.AGGATG
MGH107_Plate12_RNA_ofRRBS_5433.37_S37_L005_R1_001,-0.007662,-0.154326,0.762142,0.629251,-0.504013,-0.745628,-0.803426,AC-like,MGH107,MGH107_P8_C1_S2_L007.GTTGAG
MGH107_Plate12_RNA_ofRRBS_5433.72_S72_L005_R1_001,-0.332037,0.054306,1.009826,0.993723,-0.436964,-0.669287,-0.937791,AC-like,MGH107,MGH107_P8_E1_S3_L007.TGTAGG
MGH107_Plate12_RNA_ofRRBS_5433.77_S77_L005_R1_001,0.095566,0.046539,1.016358,0.976778,-0.526943,-1.040520,-0.976167,AC-like,MGH107,MGH107_P8_G1_S4_L007.CAAGAG
...,...,...,...,...,...,...,...,...,...,...
MGH208_P_2_H5_S105_ME_L001,-0.264061,-0.444258,-0.187746,-0.281954,0.284301,0.058129,0.117989,undifferentiated,MGH208,MGH208-2-4_S4_ME_L001.TGCTGC
MGH208_P_2_H6_S106_ME_L001,-0.706929,-0.421759,0.639534,0.791600,-0.072641,-0.076242,-0.452626,AC-like,MGH208,MGH208-2-4_S4_ME_L001.ACAGAC
MGH208_P_2_H7_S107_ME_L001,0.969103,0.100957,-0.155417,-0.216787,0.203517,0.273694,0.131818,undifferentiated,MGH208,MGH208-2-4_S4_ME_L001.AGAAGG


In [206]:
cellstates['CellAssignment'].value_counts()

undifferentiated    469
OC-like             136
AC-like             107
stem-like            32
Name: CellAssignment, dtype: int64

In [207]:
color_code = {'AC-like':'yellow','OC-like':'green', 'undifferentiated':'blue','stem-like': 'blue'}

In [208]:
colorlist = []
for tip in tree.get_tip_labels():
    cell = '.'.join(tip.split('.')[:2])
    state = cellstates[cellstates['DNAmeMatchingCell'] == cell]['CellAssignment']
    if np.size(state.values) == 0:
        colorlist.append('black')
        continue
    colorlist.append(color_code[state.values[0]])
#colorlist = [ color_code[cellstates[cellstates['DNAmeMatchingCell'] == '.'.join(tip.split('.')[:2])]['CellAssignment'].value] for tip in tree.get_tip_labels()]
tree.draw(
    tip_labels_colors=colorlist,
    use_edge_lengths=False
);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="1000.0px" viewBox="0 0 500.0 1000.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t8c701adeefff42a5b8a7544f55d5de18"> MGH107_P8_G1_S4_L007.ATCAAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_G1_S4_L007.ACAACC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_G1_S4_L007.CCTTCG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.GGTAAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_A1_S1_L007.ATCAAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_C1_S2_L007.TGTAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.CTATTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_C1_S2_L007.ATCAAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_G1_S4_L007.GGTAAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.AGAAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_G1_S4_L007.AGGATG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_G1_S4_L007.CAAGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.CGGTAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_G1_S4_L007.TGTAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_A1_S1_L007.ACAGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_C1_S2_L007.GGTAAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_A1_S1_L007.TCTCTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.TCTCTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_A1_S1_L007.CAAGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_G1_S4_L007.GTTGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.GCATTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.TATCTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_A1_S1_L007.GCATTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_G1_S4_L007.ACAGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.CCATAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_A1_S1_L007.GTTGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_C1_S2_L007.TATCTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_G1_S4_L007.TATCTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.AGGATG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_A1_S1_L007.ATCGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_A1_S1_L007.TGCTGC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_C1_S2_L007.GTTGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.ATCGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_A1_S1_L007.TGACAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_C1_S2_L007.AGGATG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_C1_S2_L007.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.TGTAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_A1_S1_L007.CCTTCG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_C1_S2_L007.CTCAGC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_A1_S1_L007.ACAACC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_G1_S4_L007.CTCAGC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_C1_S2_L007.ACAGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_C1_S2_L007.CAAGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_A1_S1_L007.TGTAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.CAAGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.ACAGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_G1_S4_L007.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_A1_S1_L007.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.ATCAAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_C1_S2_L007.GCATTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_G1_S4_L007.ACTCAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_C1_S2_L007.CCATAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH107_P8_E1_S3_L007.CATGAC.R1.fastq_bismark_bt2_pe.bismark

In [120]:
cell_cnv = pd.read_csv('../CNV_analysis/ginkgo/uploads/MGH105/CNV2', sep = '\t', header = None,
            names = ['chrom','start','end','cell','cnv'])

In [121]:
cell_cnv['cell'] = cell_cnv['cell'] + '.R1.fastq_bismark_bt2_pe.bismark.cov'

In [122]:
cell_cnv

,chrom,start,end,cell,cnv
0,chr1,1,2013121,MGH105A_P7_C1_S2_L001.GAAGTC.R1.fastq_bismark_...,1
1,chr1,1,2213136,MGH105B_P6_G1_S4_L005.GGTAAC.R1.fastq_bismark_...,1
2,chr1,1,2213136,MGH105C_P8_E1_S3_L002.ACTCAC.R1.fastq_bismark_...,1
3,chr1,1,2413679,GBM_MGH105C_C1_S2_L008.GCATTC.R1.fastq_bismark...,1
4,chr1,1,2413679,MGH105B_P6_C1_S2_L005.CATGAC.R1.fastq_bismark_...,1
...,...,...,...,...,...
4914,chrY,59141260,59373566,MGH105D_P7_C1_S2_L004.ATCAAG.R1.fastq_bismark_...,-1
4915,chrY,59141260,59373566,MGH105D_P7_C1_S2_L004.GCATTC.R1.fastq_bismark_...,-1
4916,chrY,1,59373566,MGH105D_P7_E1_S3_L004.ACAACC.R1.fastq_bismark_...,-1
4917,chrY,59141260,59373566,MGH105D_P7_E1_S3_L004.GAAGTC.R1.fastq_bismark_...,-1


In [125]:
cell_cnv = cell_cnv[cell_cnv['cell'].isin(tree.get_tip_labels())]

In [140]:
celltype = pd.read_csv('../Chaligne2021NG/Chaligne2021NG/MGH105_metadata_50.csv',index_col=0)
celltype

,# reads (x10^6),# CpGs (x10^6),CG conversion rate,Sample,Type
cell,,,,,
MGH105C_P8_E1_S3_L002.CTATTG,2.286644,0.137527,99.8,MGH105C,Immune cell
GBM_MGH105C_C1_S2_L008.GTTGAG,1.642679,0.086319,99.8,MGH105C,GBM
MGH105A_P7_G1_S4_L001.TATCTC,1.764384,0.111831,99.8,MGH105A,GBM
MGH105A_P7_G1_S4_L001.CTATTG,2.032915,0.116559,99.8,MGH105A,GBM
MGH105D_P7_A1_S1_L004.TCTCTG,2.956988,0.140904,99.8,MGH105D,GBM
MGH105B_P6_G1_S4_L005.TGACAG,2.502921,0.150394,99.8,MGH105B,GBM
MGH105A_P7_G1_S4_L001.CCTTCG,3.012091,0.070667,99.8,MGH105A,GBM
MGH105C_P8_C1_S2_L002.TCTCTG,2.393972,0.162031,99.8,MGH105C,Immune cell
GBM_MGH105C_E1_S3_L008.TGCTGC,1.479671,0.103538,99.8,MGH105C,GBM


In [149]:
md = pd.read_csv('../Chaligne2021NG/Chaligne2021NG/metadata.tsv',index_col=0, sep='\t')
md['Sample'].value_counts()

MGH142       182
MGH208       177
MGH105C      143
MGH135        96
MGH121_P1     88
MGH121_P3     81
MGH107        76
MGH64         76
MGH105A       74
MGH121_P4     67
MGH129        65
MGH201        65
MGH105B       64
MGH124        62
MGH105D       58
MGH122        51
MGH115        48
MGH45         45
MGH121_P2     39
MGH211        28
Name: Sample, dtype: int64

In [150]:
md

,# reads (x10^6),# CpGs (x10^6),CG conversion rate,Sample,Type
Cell,,,,,
MGH105A_P7_A1_S1_L001.ACAACC,1.765207,0.110823,99.8,MGH105A,GBM
MGH105A_P7_A1_S1_L001.ACTCAC,1.992651,0.130788,99.8,MGH105A,GBM
MGH105A_P7_A1_S1_L001.AGGATG,2.156402,0.125591,99.8,MGH105A,GBM
MGH105A_P7_A1_S1_L001.ATCAAG,2.646213,0.153169,99.8,MGH105A,GBM
MGH105A_P7_A1_S1_L001.ATCGAC,1.834851,0.121873,99.8,MGH105A,GBM
...,...,...,...,...,...
MGH201-1-4_S8_ME_L001.CTATTG,2.770118,0.074693,99.5,MGH201,IDH-MUT
MGH201-1-4_S8_ME_L001.GAAGTC,3.821266,0.120119,99.8,MGH201,IDH-MUT
MGH201-1-4_S8_ME_L001.GCATTC,1.861971,0.060702,99.8,MGH201,IDH-MUT


In [136]:
#segcopy = pd.read_csv('../CNV_analysis/ginkgo/uploads/MGH105/SegCopy',sep = '\t')
segcopy = pd.read_csv('segcopy.txt',sep = '\t')
segcopy 

,CHR,START,END,GBM_MGH105C_A1_S1_L008.CCTTCG,GBM_MGH105C_C1_S2_L008.ATCGAC,GBM_MGH105C_C1_S2_L008.GCATTC,GBM_MGH105C_C1_S2_L008.GTTGAG,GBM_MGH105C_E1_S3_L008.TGCTGC,GBM_MGH105C_G1_S4_L008.GTGAGG,GBM_MGH105C_G1_S4_L008.TGCTGC,...,MGH105C_P8_G1_S4_L002.GCATTC,MGH105D_P7_A1_S1_L004.ACTCAC,MGH105D_P7_A1_S1_L004.CATGAC,MGH105D_P7_A1_S1_L004.CCATAG,MGH105D_P7_A1_S1_L004.TCTCTG,MGH105D_P7_C1_S2_L004.ATCAAG,MGH105D_P7_C1_S2_L004.GCATTC,MGH105D_P7_E1_S3_L004.ACAACC,MGH105D_P7_E1_S3_L004.GAAGTC,MGH105D_P7_E1_S3_L004.GTTGAG
0,chr1,1,1246396,6,2,3,3,3,2,6,...,2,2,2,2,2,2,2,2,2,2
1,chr1,1246397,1833664,6,2,3,3,3,2,6,...,2,2,2,2,2,2,2,2,2,2
2,chr1,1833665,2345598,6,2,3,3,3,2,6,...,2,2,2,2,2,2,2,2,2,2
3,chr1,2345599,2946628,6,2,3,3,3,2,6,...,2,2,2,2,2,2,2,2,2,2
4,chr1,2946629,3452945,6,2,3,3,3,2,6,...,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5358,chrY,21426519,21966606,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5359,chrY,21966607,22729050,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5360,chrY,22729051,23300405,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5361,chrY,23300406,23986320,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [137]:
cn = segcopy.groupby(by='CHR').mean().round()#.astype(int)#.round()
cn

,START,END,GBM_MGH105C_A1_S1_L008.CCTTCG,GBM_MGH105C_C1_S2_L008.ATCGAC,GBM_MGH105C_C1_S2_L008.GCATTC,GBM_MGH105C_C1_S2_L008.GTTGAG,GBM_MGH105C_E1_S3_L008.TGCTGC,GBM_MGH105C_G1_S4_L008.GTGAGG,GBM_MGH105C_G1_S4_L008.TGCTGC,GBM_MGH105C_G1_S4_L008.TGTAGG,...,MGH105C_P8_G1_S4_L002.GCATTC,MGH105D_P7_A1_S1_L004.ACTCAC,MGH105D_P7_A1_S1_L004.CATGAC,MGH105D_P7_A1_S1_L004.CCATAG,MGH105D_P7_A1_S1_L004.TCTCTG,MGH105D_P7_C1_S2_L004.ATCAAG,MGH105D_P7_C1_S2_L004.GCATTC,MGH105D_P7_E1_S3_L004.ACAACC,MGH105D_P7_E1_S3_L004.GAAGTC,MGH105D_P7_E1_S3_L004.GTTGAG
CHR,,,,,,,,,,,,,,,,,,,,,
chr1,123695110.0,124284354.0,6.0,2.0,3.0,3.0,3.0,2.0,6.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
chr10,68951149.0,69502102.0,4.0,1.0,2.0,2.0,2.0,1.0,4.0,1.0,...,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
chr11,67956068.0,68498262.0,5.0,2.0,3.0,3.0,2.0,2.0,5.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
chr12,67885105.0,68418378.0,6.0,2.0,3.0,3.0,3.0,2.0,6.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
chr13,66739950.0,67362489.0,6.0,2.0,3.0,4.0,3.0,2.0,6.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
chr14,63384997.0,64020200.0,6.0,2.0,3.0,3.0,3.0,2.0,5.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
chr15,62457002.0,63140544.0,4.0,2.0,3.0,3.0,3.0,2.0,4.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
chr16,46496825.0,47128675.0,5.0,2.0,2.0,3.0,3.0,2.0,5.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
chr17,41607895.0,42167861.0,6.0,2.0,4.0,3.0,3.0,2.0,6.0,2.0,...,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [138]:
chr6 = cn.loc['chr6',:][2:]
dict_chr6 = {}
for i, n in zip(chr6.index, chr6):
    if n < 2: 
        dict_chr6[i] = 'blue' #'loss'
    elif n >= 3:
        dict_chr6[i] = 'red'  #'gain'
    else:
        dict_chr6[i] = 'black'#'normal'

#color by copy number of the whole chrom
colorlist = [ dict_chr6['.'.join(tip.split('.')[:2])] for tip in tree.get_tip_labels() ]
tree.draw(
    tip_labels_colors=colorlist,
    use_edge_lengths=False
);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="720.0px" viewBox="0 0 500.0 720.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t275dc6054dcc48c29e3d9a2bbe91dc1d"> MGH105D_P7_A1_S1_L004.TCTCTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_E1_S3_L005.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_A1_S1_L004.CATGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_C1_S2_L004.ATCAAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_G1_S4_L005.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_G1_S4_L001.GGTAAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_C1_S2_L004.GCATTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_G1_S4_L005.ACTCAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_A1_S1_L005.ACAACC.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_C1_S2_L008.GTTGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_C1_S2_L005.CATGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_E1_S3_L005.GAAGTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_G1_S4_L005.CATGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_G1_S4_L001.CCTTCG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_E1_S3_L005.ATCAAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_G1_S4_L005.TGACAG.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_C1_S2_L008.ATCGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_E1_S3_L004.ACAACC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_C1_S2_L001.TCTCTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_G1_S4_L005.GGTAAC.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_G1_S4_L008.TGCTGC.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_E1_S3_L008.TGCTGC.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_A1_S1_L008.CCTTCG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_E1_S3_L004.GAAGTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_G1_S4_L001.CTATTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_A1_S1_L001.CAAGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_E1_S3_L004.GTTGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_C1_S2_L001.GAAGTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_A1_S1_L004.CCATAG.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_G1_S4_L008.TGTAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_C1_S2_L005.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_G1_S4_L008.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_A1_S1_L005.ACAGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_A1_S1_L001.ACTCAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_C1_S2_L001.GGTAAC.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_C1_S2_L008.GCATTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_E1_S3_L005.TATCTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_A1_S1_L004.ACTCAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_A1_S1_L005.ATCGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_G1_S4_L001.TATCTC.R1.fastq_bismark_bt2_pe.bismark.cov

In [60]:
#color by copy number of the whole chrom
colorlist = [ dict_chr6[tip] for tip in tree.get_tip_labels() ]
tree.draw(
    tip_labels_colors=colorlist,
    use_edge_lengths=False
);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="395.0px" height="900.0px" viewBox="0 0 395.0 900.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t5e9e37cfbd8e4613ac0b4d252670c049"> MGH105C_P8_G1_S4_L002.GCATTC MGH105C_P8_G1_S4_L002.CATGAC MGH105C_P8_E1_S3_L002.GTGAGG GBM_MGH105C_G1_S4_L008.TGCTGC GBM_MGH105C_A1_S1_L008.CCTTCG MGH105B_P6_A1_S1_L005.ACAACC GBM_MGH105C_C1_S2_L008.GTTGAG GBM_MGH105C_E1_S3_L008.TGCTGC MGH105A_P7_G1_S4_L001.CCTTCG MGH105A_P7_G1_S4_L001.TATCTC MGH105A_P7_G1_S4_L001.CTATTG MGH105A_P7_C1_S2_L001.GGTAAC GBM_MGH105C_C1_S2_L008.GCATTC MGH105A_P7_A1_S1_L001.ACTCAC MGH105A_P7_C1_S2_L001.TCTCTG MGH105A_P7_C1_S2_L001.GAAGTC MGH105B_P6_E1_S3_L005.GTGAGG MGH105A_P7_G1_S4_L001.GGTAAC MGH105A_P7_A1_S1_L001.CAAGAG MGH105D_P7_E1_S3_L004.GAAGTC MGH105D_P7_E1_S3_L004.GTTGAG GBM_MGH105C_G1_S4_L008.TGTAGG MGH105B_P6_G1_S4_L005.TGACAG MGH105D_P7_C1_S2_L004.GCATTC GBM_MGH105C_G1_S4_L008.GTGAGG GBM_MGH105C_C1_S2_L008.ATCGAC MGH105D_P7_C1_S2_L004.ATCAAG MGH105B_P6_E1_S3_L005.ATCAAG MGH105B_P6_A1_S1_L005.ACAGAC MGH105C_P8_E1_S3_L002.ACTCAC MGH105C_P8_C1_S2_L002.TCTCTG MGH105C_P8_E1_S3_L002.CTATTG MGH105B_P6_G1_S4_L005.CATGAC MGH105C_P8_A1_S1_L002.AGGATG MGH105C_P8_A1_S1_L002.TATCTC MGH105C_P8_C1_S2_L002.GGTAAC MGH105B_P6_C1_S2_L005.GTGAGG MGH105B_P6_A1_S1_L005.ATCGAC MGH105B_P6_E1_S3_L005.TATCTC MGH105B_P6_C1_S2_L005.ACAGAC MGH105D_P7_A1_S1_L004.TCTCTG MGH105B_P6_G1_S4_L005.GGTAAC MGH105D_P7_A1_S1_L004.ACTCAC MGH105B_P6_G1_S4_L005.ACTCAC MGH105B_P6_E1_S3_L005.GAAGTC MGH105B_P6_G1_S4_L005.GTGAGG MGH105D_P7_A1_S1_L004.CCATAG MGH105D_P7_E1_S3_L004.ACAACC MGH105D_P7_A1_S1_L004.CATGAC MGH105B_P6_C1_S2_L005.CATGAC

### color by cell type

In [151]:
cellstates  = pd.read_csv('../Chaligne2021NG/WTcellstates.csv', index_col = 0)
cellstates

,MESlike2,MESlike1,Aclike,OPClike,NPClike1,NPClike2,G1S,G2M,CellAssignment,Sample,DNAmeMatchingCell
Cell,,,,,,,,,,,
MGH105A_P7_A3_S3_L003_R1_001,0.002211,-0.723862,0.554145,0.011484,-0.607437,-0.510248,-0.143740,-0.185441,AC,MGH105A,MGH105A_P7_A1_S1_L001.AGGATG
MGH105A_P7_B11_S23_L003_R1_001,-0.560369,0.175751,1.323466,0.107158,-0.318620,0.352542,0.163651,-0.088464,AC,MGH105A,MGH105A_P7_A1_S1_L001.GGTAAC
MGH105A_P7_B12_S24_L003_R1_001,-0.037159,0.130776,1.671545,0.548509,-0.251249,0.195576,0.079633,-0.113761,AC,MGH105A,MGH105A_P7_A1_S1_L001.TGTAGG
MGH105A_P7_B5_S17_L003_R1_001,-0.699869,-0.726169,1.520971,0.717245,-0.055908,-0.141890,-0.481268,-0.220056,AC,MGH105A,MGH105A_P7_A1_S1_L001.TGCTGC
MGH105A_P7_C10_S34_L003_R1_001,-1.143335,-0.361016,1.074859,0.148747,-0.034902,0.112690,0.578794,-0.286407,AC,MGH105A,MGH105A_P7_C1_S2_L001.CTCAGC
...,...,...,...,...,...,...,...,...,...,...,...
MGH211_P13_F12_S72_L007_R1_001,-0.918219,0.041154,-0.133389,0.143759,1.202246,2.070784,0.018044,-0.363653,NPC.2,MGH211,NaN
MGH211_P13_G12_S84_L007_R1_001,-1.217079,-1.330635,-1.011145,-0.237150,0.647991,1.421304,-0.120602,-0.384374,NPC.2,MGH211,NaN
MGH211_P13_G3_S75_L007_R1_001,-0.649483,-0.999679,-0.827158,0.043493,0.809762,1.049299,-0.096195,-0.473733,NPC.2,MGH211,NaN


In [156]:
cellstates[cellstates['Sample'].isin(['MGH105A'])]['CellAssignment'].value_counts()

AC       22
NPC.1    14
MES.1    12
MES.2    10
G1.S      3
G2.M      3
OPC       3
NPC.2     1
Name: CellAssignment, dtype: int64

In [160]:
color_code = {'MGH105A':'mediumseagreen','MGH105B':"#d6557c",'MGH105C':'black','MGH105D':'blue'}
colorlist = [ color_code[celltype.loc['.'.join(tip.split('.')[:2]),'Sample']] for tip in tree.get_tip_labels() ]
tree.draw(
    tip_labels_colors=colorlist,
    use_edge_lengths=False
);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="720.0px" viewBox="0 0 500.0 720.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t3065b4b5aa8f4fae8bfda6c18054689e"> MGH105D_P7_A1_S1_L004.TCTCTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_E1_S3_L005.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_A1_S1_L004.CATGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_C1_S2_L004.ATCAAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_G1_S4_L005.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_G1_S4_L001.GGTAAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_C1_S2_L004.GCATTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_G1_S4_L005.ACTCAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_A1_S1_L005.ACAACC.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_C1_S2_L008.GTTGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_C1_S2_L005.CATGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_E1_S3_L005.GAAGTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_G1_S4_L005.CATGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_G1_S4_L001.CCTTCG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_E1_S3_L005.ATCAAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_G1_S4_L005.TGACAG.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_C1_S2_L008.ATCGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_E1_S3_L004.ACAACC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_C1_S2_L001.TCTCTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_G1_S4_L005.GGTAAC.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_G1_S4_L008.TGCTGC.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_E1_S3_L008.TGCTGC.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_A1_S1_L008.CCTTCG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_E1_S3_L004.GAAGTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_G1_S4_L001.CTATTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_A1_S1_L001.CAAGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_E1_S3_L004.GTTGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_C1_S2_L001.GAAGTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_A1_S1_L004.CCATAG.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_G1_S4_L008.TGTAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_C1_S2_L005.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_G1_S4_L008.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_A1_S1_L005.ACAGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_A1_S1_L001.ACTCAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_C1_S2_L001.GGTAAC.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_C1_S2_L008.GCATTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_E1_S3_L005.TATCTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_A1_S1_L004.ACTCAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_A1_S1_L005.ATCGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_G1_S4_L001.TATCTC.R1.fastq_bismark_bt2_pe.bismark.cov

In [184]:
cellstates[cellstates['DNAmeMatchingCell'] == 'MGH105D_P7_A1_S1_L004.TCTCTG']['CellAssignment'].values[0]

'NPC.1'

In [187]:
color_code = {'AC':'yellow','OPC':'green', 'NPC.1':'blue','NPC.2':'blue',
              'MES.1':'red','MES.2':'red','G1.S':'grey','G2.M':'grey'}

In [196]:
colorlist

['blue', 'red', 'grey', 'yellow']

In [198]:
colorlist = []
for tip in tree.get_tip_labels():
    cell = '.'.join(tip.split('.')[:2])
    state = cellstates[cellstates['DNAmeMatchingCell'] == cell]['CellAssignment']
    if np.size(state.values) == 0:
        colorlist.append('black')
        continue
    colorlist.append(color_code[state.values[0]])
#colorlist = [ color_code[cellstates[cellstates['DNAmeMatchingCell'] == '.'.join(tip.split('.')[:2])]['CellAssignment'].value] for tip in tree.get_tip_labels()]
tree.draw(
    tip_labels_colors=colorlist,
    use_edge_lengths=False
);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="720.0px" viewBox="0 0 500.0 720.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t2b4e57025c7848eda4d56a5246afd58a"> MGH105D_P7_A1_S1_L004.TCTCTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_E1_S3_L005.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_A1_S1_L004.CATGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_C1_S2_L004.ATCAAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_G1_S4_L005.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_G1_S4_L001.GGTAAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_C1_S2_L004.GCATTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_G1_S4_L005.ACTCAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_A1_S1_L005.ACAACC.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_C1_S2_L008.GTTGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_C1_S2_L005.CATGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_E1_S3_L005.GAAGTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_G1_S4_L005.CATGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_G1_S4_L001.CCTTCG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_E1_S3_L005.ATCAAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_G1_S4_L005.TGACAG.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_C1_S2_L008.ATCGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_E1_S3_L004.ACAACC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_C1_S2_L001.TCTCTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_G1_S4_L005.GGTAAC.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_G1_S4_L008.TGCTGC.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_E1_S3_L008.TGCTGC.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_A1_S1_L008.CCTTCG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_E1_S3_L004.GAAGTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_G1_S4_L001.CTATTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_A1_S1_L001.CAAGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_E1_S3_L004.GTTGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_C1_S2_L001.GAAGTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_A1_S1_L004.CCATAG.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_G1_S4_L008.TGTAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_C1_S2_L005.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_G1_S4_L008.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_A1_S1_L005.ACAGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_A1_S1_L001.ACTCAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_C1_S2_L001.GGTAAC.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_C1_S2_L008.GCATTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_E1_S3_L005.TATCTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_A1_S1_L004.ACTCAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_A1_S1_L005.ATCGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_G1_S4_L001.TATCTC.R1.fastq_bismark_bt2_pe.bismark.cov

In [194]:
state.values

array([], dtype=object)

In [143]:
chr7 = cell_cnv.loc[cell_cnv['chrom']=='chr7',:]
# index of -1, index of 1, intersection of both: del_dup
# only -1: del, only 1: dup, none: normal 
g = chr7.groupby(by='cnv')
gain = set(g.get_group(1)['cell'])
loss = set(g.get_group(-1)['cell'])
both = gain.intersection(loss)
chr7_cnv = {'gain':gain-both, 'loss': loss-both, 'both': both, 'normal': set(tree.get_tip_labels())-gain - loss}
chr7_cnv

 #[print(x, y) for x in iter1 for y in iter2] 
chr7_cnv = { cell: cnv for cnv, l in chr7_cnv.items() for cell in l}

In [35]:
#colorlist
#set(tree.get_tip_labels())-gain - loss
chr7_cnv

{'MGH105B_P6_C1_S2_L005.ACAGAC': 'gain',
 'MGH105D_P7_E1_S3_L004.GAAGTC': 'gain',
 'MGH105C_P8_C1_S2_L002.TCTCTG': 'gain',
 'GBM_MGH105C_A1_S1_L008.CCTTCG': 'gain',
 'MGH105C_P8_A1_S1_L002.AGGATG': 'gain',
 'GBM_MGH105C_C1_S2_L008.GTTGAG': 'gain',
 'MGH105C_P8_E1_S3_L002.CTATTG': 'gain',
 'MGH105B_P6_C1_S2_L005.GTGAGG': 'gain',
 'MGH105A_P7_C1_S2_L001.TCTCTG': 'gain',
 'MGH105D_P7_C1_S2_L004.GCATTC': 'gain',
 'MGH105B_P6_G1_S4_L005.GTGAGG': 'gain',
 'MGH105B_P6_G1_S4_L005.CATGAC': 'gain',
 'MGH105D_P7_C1_S2_L004.ATCAAG': 'gain',
 'MGH105A_P7_C1_S2_L001.GAAGTC': 'gain',
 'MGH105B_P6_G1_S4_L005.GGTAAC': 'gain',
 'MGH105B_P6_E1_S3_L005.GTGAGG': 'gain',
 'MGH105A_P7_G1_S4_L001.CTATTG': 'gain',
 'MGH105B_P6_A1_S1_L005.ACAGAC': 'gain',
 'MGH105D_P7_A1_S1_L004.TCTCTG': 'gain',
 'GBM_MGH105C_E1_S3_L008.TGCTGC': 'gain',
 'MGH105B_P6_A1_S1_L005.ACAACC': 'gain',
 'MGH105B_P6_E1_S3_L005.GAAGTC': 'gain',
 'MGH105C_P8_A1_S1_L002.TATCTC': 'gain',
 'MGH105D_P7_E1_S3_L004.ACAACC': 'gain',
 'MGH105D_P7_

In [144]:
color_code = {'both':'mediumseagreen','loss':"#d6557c", 'gain':"#5384a3",'normal':'black'}
colorlist = [color_code[chr7_cnv[tip]] for tip in tree.get_tip_labels()]
tree.draw(
    #tip_labels_align=True,
    tip_labels_colors=colorlist,
    use_edge_lengths=False
);
#gain: blue, loss: red:  both: green 

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="720.0px" viewBox="0 0 500.0 720.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tda49f09b89a34b7bb3bbd73a1253f997"> MGH105D_P7_A1_S1_L004.TCTCTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_E1_S3_L005.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_A1_S1_L004.CATGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_C1_S2_L004.ATCAAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_G1_S4_L005.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_G1_S4_L001.GGTAAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_C1_S2_L004.GCATTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_G1_S4_L005.ACTCAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_A1_S1_L005.ACAACC.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_C1_S2_L008.GTTGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_C1_S2_L005.CATGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_E1_S3_L005.GAAGTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_G1_S4_L005.CATGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_G1_S4_L001.CCTTCG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_E1_S3_L005.ATCAAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_G1_S4_L005.TGACAG.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_C1_S2_L008.ATCGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_E1_S3_L004.ACAACC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_C1_S2_L001.TCTCTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_G1_S4_L005.GGTAAC.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_G1_S4_L008.TGCTGC.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_E1_S3_L008.TGCTGC.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_A1_S1_L008.CCTTCG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_E1_S3_L004.GAAGTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_G1_S4_L001.CTATTG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_A1_S1_L001.CAAGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_E1_S3_L004.GTTGAG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_C1_S2_L001.GAAGTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_A1_S1_L004.CCATAG.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_G1_S4_L008.TGTAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_C1_S2_L005.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_G1_S4_L008.GTGAGG.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_A1_S1_L005.ACAGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_A1_S1_L001.ACTCAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_C1_S2_L001.GGTAAC.R1.fastq_bismark_bt2_pe.bismark.cov GBM_MGH105C_C1_S2_L008.GCATTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_E1_S3_L005.TATCTC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105D_P7_A1_S1_L004.ACTCAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105B_P6_A1_S1_L005.ATCGAC.R1.fastq_bismark_bt2_pe.bismark.cov MGH105A_P7_G1_S4_L001.TATCTC.R1.fastq_bismark_bt2_pe.bismark.cov

In [37]:
chr10 = cell_cnv.loc[cell_cnv['chrom']=='chr6',:]
# index of -1, index of 1, intersection of both: del_dup
# only -1: del, only 1: dup, none: normal 
g = chr10.groupby(by='cnv')
gain = set(g.get_group(1)['cell'])
loss = set(g.get_group(-1)['cell'])
both = gain.intersection(loss)
chr10_cnv = {'gain':gain-both, 'loss': loss-both, 'both': both, 'normal': set(tree.get_tip_labels())-gain - loss}
 #[print(x, y) for x in iter1 for y in iter2] 
chr10_cnv = { cell: cnv for cnv, l in chr10_cnv.items() for cell in l}

In [38]:
color_code = {'both':'mediumseagreen','loss':"#d6557c", 'gain':"#5384a3",'normal':'black'}
colorlist = [color_code[chr10_cnv[tip]] for tip in tree.get_tip_labels()]
tree.draw(
    #tip_labels_align=True,
    tip_labels_colors=colorlist,
    use_edge_lengths=False
);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="395.0px" height="900.0px" viewBox="0 0 395.0 900.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t007923649f8e4029bbdb54947006dca0"> MGH105C_P8_G1_S4_L002.GCATTC MGH105C_P8_G1_S4_L002.CATGAC MGH105C_P8_E1_S3_L002.GTGAGG GBM_MGH105C_G1_S4_L008.TGCTGC GBM_MGH105C_A1_S1_L008.CCTTCG MGH105B_P6_A1_S1_L005.ACAACC GBM_MGH105C_C1_S2_L008.GTTGAG GBM_MGH105C_E1_S3_L008.TGCTGC MGH105A_P7_G1_S4_L001.CCTTCG MGH105A_P7_G1_S4_L001.TATCTC MGH105A_P7_G1_S4_L001.CTATTG MGH105A_P7_C1_S2_L001.GGTAAC GBM_MGH105C_C1_S2_L008.GCATTC MGH105A_P7_A1_S1_L001.ACTCAC MGH105A_P7_C1_S2_L001.TCTCTG MGH105A_P7_C1_S2_L001.GAAGTC MGH105B_P6_E1_S3_L005.GTGAGG MGH105A_P7_G1_S4_L001.GGTAAC MGH105A_P7_A1_S1_L001.CAAGAG MGH105D_P7_E1_S3_L004.GAAGTC MGH105D_P7_E1_S3_L004.GTTGAG GBM_MGH105C_G1_S4_L008.TGTAGG MGH105B_P6_G1_S4_L005.TGACAG MGH105D_P7_C1_S2_L004.GCATTC GBM_MGH105C_G1_S4_L008.GTGAGG GBM_MGH105C_C1_S2_L008.ATCGAC MGH105D_P7_C1_S2_L004.ATCAAG MGH105B_P6_E1_S3_L005.ATCAAG MGH105B_P6_A1_S1_L005.ACAGAC MGH105C_P8_E1_S3_L002.ACTCAC MGH105C_P8_C1_S2_L002.TCTCTG MGH105C_P8_E1_S3_L002.CTATTG MGH105B_P6_G1_S4_L005.CATGAC MGH105C_P8_A1_S1_L002.AGGATG MGH105C_P8_A1_S1_L002.TATCTC MGH105C_P8_C1_S2_L002.GGTAAC MGH105B_P6_C1_S2_L005.GTGAGG MGH105B_P6_A1_S1_L005.ATCGAC MGH105B_P6_E1_S3_L005.TATCTC MGH105B_P6_C1_S2_L005.ACAGAC MGH105D_P7_A1_S1_L004.TCTCTG MGH105B_P6_G1_S4_L005.GGTAAC MGH105D_P7_A1_S1_L004.ACTCAC MGH105B_P6_G1_S4_L005.ACTCAC MGH105B_P6_E1_S3_L005.GAAGTC MGH105B_P6_G1_S4_L005.GTGAGG MGH105D_P7_A1_S1_L004.CCATAG MGH105D_P7_E1_S3_L004.ACAACC MGH105D_P7_A1_S1_L004.CATGAC MGH105B_P6_C1_S2_L005.CATGAC

In [141]:
chr6 = cell_cnv.loc[cell_cnv['chrom']=='chr19',:]
# index of -1, index of 1, intersection of both: del_dup
# only -1: del, only 1: dup, none: normal 
g = chr6.groupby(by='cnv')
gain = set(g.get_group(1)['cell'])
loss = set(g.get_group(-1)['cell'])
both = gain.intersection(loss)
chr6_cnv = {'gain':gain-both, 'loss': loss-both, 'both': both, 'normal': set(tree.get_tip_labels())-gain - loss}
 #[print(x, y) for x in iter1 for y in iter2] 
chr6_cnv = { cell: cnv for cnv, l in chr6_cnv.items() for cell in l}

color_code = {'both':'mediumseagreen','loss':"#d6557c", 'gain':"#5384a3",'normal':'black'}
colorlist = [color_code[chr6_cnv[tip]] for tip in tree.get_tip_labels()]
tree.draw(
    #tip_labels_align=True,
    tip_labels_colors=colorlist,
    use_edge_lengths=False
);
#gain: blue, loss: red:  both: green 

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="395.0px" height="900.0px" viewBox="0 0 395.0 900.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tf9709fa2964a4da19b067900740cb47d"> CW_2209_1-CW_2209_2-D10-AD006 CW_2209_1-CW_2209_2-B10-AD002 CW_2209_1-CW_2209_2-E10-AD008 CW_2209_1-CW_2209_2-H10-AD007 CW_2209_1-CW_2209_2-D10-AD012 CW_2209_1-CW_2209_2-A10-AD008 CW_2209_1-CW_2209_2-H10-AD001 CW_2209_1-CW_2209_2-A11-AD004 CW_2209_1-CW_2209_2-A11-AD010 CW_2209_1-CW_2209_2-F10-AD001 CW_2209_1-CW_2209_2-B11-AD001 CW_2209_1-CW_2209_2-A10-AD012 CW_2209_1-CW_2209_2-H10-AD012 CW_2209_1-CW_2209_2-B11-AD002 CW_2209_1-CW_2209_2-F10-AD012 CW_2209_1-CW_2209_2-A11-AD002 CW_2209_1-CW_2209_2-B10-AD007 CW_2209_1-CW_2209_2-C10-AD002 CW_2209_1-CW_2209_2-D10-AD001 CW_2209_1-CW_2209_2-B10-AD004 CW_2209_1-CW_2209_2-A10-AD004 CW_2209_1-CW_2209_2-A11-AD006 CW_2209_1-CW_2209_2-A11-AD007 CW_2209_1-CW_2209_2-C10-AD012 CW_2209_1-CW_2209_2-F10-AD002 CW_2209_1-CW_2209_2-C10-AD007 CW_2209_1-CW_2209_2-H10-AD006 CW_2209_1-CW_2209_2-H10-AD004 CW_2209_1-CW_2209_2-G10-AD010 CW_2209_1-CW_2209_2-H10-AD008 CW_2209_1-CW_2209_2-D10-AD007 CW_2209_1-CW_2209_2-A11-AD012 CW_2209_1-CW_2209_2-C10-AD004 CW_2209_1-CW_2209_2-C10-AD010 CW_2209_1-CW_2209_2-F10-AD007 CW_2209_1-CW_2209_2-A11-AD008 CW_2209_1-CW_2209_2-D10-AD002 CW_2209_1-CW_2209_2-H10-AD010 CW_2209_1-CW_2209_2-E10-AD010 CW_2209_1-CW_2209_2-E10-AD006 CW_2209_1-CW_2209_2-E10-AD002 CW_2209_1-CW_2209_2-E10-AD007 CW_2209_1-CW_2209_2-F10-AD006 CW_2209_1-CW_2209_2-A11-AD001 CW_2209_1-CW_2209_2-C10-AD001 CW_2209_1-CW_2209_2-F10-AD008 CW_2209_1-CW_2209_2-B10-AD010 CW_2209_1-CW_2209_2-B10-AD008 CW_2209_1-CW_2209_2-G10-AD002 CW_2209_1-CW_2209_2-G10-AD006